# Análisis de riesgo de una compañía aseguradora

- Instituto Tecnológico y de Estudios Superiores de Monterrey

| Alumno  | Matrícula |
| :-----: |  :-----:  |
| Verónica Victoria García De la Fuente | A00830383 |
| Emily Rebeca Méndez Cruz              | A00830768 |
| Daniel de Zamacona Madero             | A01570576 |
| Eugenio Santiesteban Zolezzi          | A01720932 |
| Juan Pablo Echeagaray González        | A00830646 |

- Profesores:
  - Dr. Fernando Elizalde Ramírez
  - Dr. Jaime Eduardo Martínez Sánchez
- Optimización estocástica
- 10 de septiembre del 2022

## Dependencias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import scipy.stats as st

from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.diagnostic import lilliefors
from statsmodels.stats.gof import chisquare

from scipy.stats import pearsonr

from matplotlib import rc

rc('font',**{'family':'serif','serif':['Times']})
rc('text', usetex=True)


## Lectura y procesamiento de datos

In [2]:
df = pd.read_csv('data/insurance-data.csv', encoding='latin-1')
print(df.shape)
df.head()


(27121, 8)


,Fecha del Siniestro,Tipo de auto,Modelo,Monto del siniestro,Aplica cobertura,Deducible,Reclamo de no-Cobertura,Pérdida total
0,01/12/2020,Deportivo,2017,200000,Si,NaN,Si,Si
1,12/05/2020,Austero,2016,100000,Si,NaN,Si,Si
2,11/01/2020,compacto,2017,150000,Si,NaN,Si,Si
3,21/12/2020,Subcompacto,2017,70000,Si,NaN,Si,Si
4,30/05/2020,Subcompacto,2019,90000,Si,NaN,Si,Si


In [ ]:
df.dtypes


Una buena práctica con el manejo de datos es asegurarnos de que sean representados por un _tipo_ de dato acorde al valor que representa la variable, esto ayudará a realizar operaciones con ellos, así como minimizar el espacio en memoria que ocupan. Cambiaremos también el nombre de las columnas para que sea más sencillo el proceso de desarrollo del modelo.

In [ ]:
column_names = {'Fecha del Siniestro': 'date',
                'Tipo de auto': 'car_type',
                'Modelo': 'model',
                'Monto del siniestro': 'amount',
                'Aplica cobertura': 'coverage', 
                'Deducible': 'deductible', 
                'Reclamo de no-Cobertura': 'no_claim',
                'Pérdida total': 'total_loss'}

df.rename(columns=column_names, inplace=True)
df.head()


In [ ]:
df.car_type.value_counts()


De nuevo continuaremos con la tarea de limpieza de datos, tratando valores nulos, convirtiendo variables del tipo booleanas a una representación adecuada, y cambiando el formato de la columna que describe el momento en el que ocurrió un siniestro. Estas transformaciones facilitarán el proceso de análisis y visualización de los mismos datos.

In [ ]:
car_map = {'Deportivo': 0, 
        'Austero': 1, 
        'compacto': 2, 
        'Subcompacto': 3,
        'Camioneta': 4,
        'De Lujo': 5
}
df['car_type'] = df['car_type'].map(car_map)
df['car_type'] = df['car_type'].fillna(99)
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
# Codificando variables booleanas a 0 y 1
df['coverage'] = df['coverage'].map({'Si': 1, 'No': 0})
df['no_claim'] = df['no_claim'].map({'Si': 1, 'No': 0})
df['total_loss'] = df['total_loss'].map({'Si': 1, 'No': 0})
df.head()


Un último paso de limpieza que garantiza que nuestro proyecto sea válido, es solamente considerar aquellos casos en los que si aplicó el seguro; este trabajo no analizará aquellos casos en los que el seguro de los clientes de la empresa no haya podido actuar por el motivo que sea.

In [ ]:
df_use = df[df['coverage'] == 1]


In [ ]:
df_use.head()


## Estimación de parámetros

La siguiente etapa del proyecto será analizar los montos pagados en cada siniestro así como la frecuencia con la que estos ocurren. Es de nuestro interés conocer las distribuciones tanto los respectivos parámetros que modelan estos fenómenos.

### Sobre el monto de los siniestros

In [ ]:
sns.displot(df_use['amount'], height=6, aspect=8/6, kde=True)
plt.xlabel('Monto del siniestro')
plt.ylabel('Frecuencia')
plt.title('Distribución del monto de los siniestros');
plt.show()


Una primera visualización de los montos de los siniestros nos haría pensar que una distribución exponencial podría ser bien ajustada a nuestros datos muestrales. Empíricamente podemos observar un pico al comienzo de la gráfica, y un continuo descenso hacia frecuencias de 0 para los montos más elevados registrados en la base de datos.

#### Prueba de hipótesis sobre la distribución del monto de los siniestros

<!-- @article{lilliefors1969kolmogorov,
  title={On the Kolmogorov-Smirnov test for the exponential distribution with mean unknown},
  author={Lilliefors, Hubert W},
  journal={Journal of the American Statistical Association},
  volume={64},
  number={325},
  pages={387--389},
  year={1969},
  publisher={Taylor \& Francis}
} -->

Realizaremos una prueba de hipótesis para ver si existe información estadística suficiente como para aseverar que el monto de los reclamos sigue una distribución exponencial $X \sim \text{Exp}(\lambda = 32198.560688)$, realizaremos esta prueba con un nivel de significancia $\alpha = 0.01$. Como método de prueba usaremos Lilliefors, propuesto en su artículo XXXX:

$$\begin{gather*}
    H_o: X \sim \text{Exp}(\lambda) \\
    H_a: X \not \sim \text{Exp}(\lambda)
\end{gather*}$$

> Lilliefors, H. W. (1969). On the Kolmogorov-Smirnov test for the exponential distribution with mean unknown. Journal of the American Statistical Association, 64(325), 387-389.

In [ ]:
stat, p_val = lilliefors(df_use['amount'], dist='exp', pvalmethod='approx')
print(f'Test Statistic: {stat}, p-value: {p_val}')


El `p-value` obtenido no es menor al nivel de significancia que hemos establecido para nuestra prueba, por lo que hemos fallado el rechazo de la hipótesis nula. De ahora en adelante supondremos que los montos de los siniestros siguen una distribución exponencial.
$$\begin{equation}
    X \sim \text{Exp}(\lambda)
\end{equation}$$

#### Prueba de hipótesis sobre la media del monto

Sabiendo que la distribución exponencial modela esta variable aleatoria, queremos ahora saber el parámetro que mejor se ajuste a nuestros datos. Dado que el parámetro $\lambda$ de la distribución exponencial es equivalente a la media de la población, supondremos que el parámetro puede ser aproximado por la media muestral de esta variable.

In [ ]:
mean_amount = df_use['amount'].describe()['mean']
print(f'Lambda = {mean_amount}')


Realizamos ahora una prueba de hipótesis con una signifcancia $\alpha = 0.05$ para la media de los montos de los siniestros:
$$\begin{gather*}
    H_o: \lambda = \bar{\lambda} \\
    H_a: \lambda \neq \bar{\lambda}
\end{gather*}$$

Como tenemos una muestra con más de 27,000 valores, podremos suponer que por el Teorema Central del Límite la media sigue una distribución normal; así que una prueba de hipótesis `t-student` bastará para comprobar nuestro supuesto.

In [ ]:
st.ttest_1samp(df_use['amount'], mean_amount)


El `p-value` que hemos obtenido no es menor al nivel de significancia establecido, por lo que no podemos rechazar la hipótesis nula. Supondremos que la media muestral de nuestra base de datos es representativa de la media poblacional.

Ahora podemos modelar la variable aleatoria de los montos como:
$$\begin{equation}
    X \sim \text{Exp}(\lambda = 30,771.375)
\end{equation}$$

#### Intervalo de confianza para la media de los siniestros

Finalmente, es de nuestro interés conocer un intervalo de confianza para nuestra estimación; seguiremos la misma convención de usar un nivel de significancia del $0.05$, y aprovecharemos el Teorema Central del Límite aplicado a medias para calcular nuestro intervalo de confianza como:
$$\begin{equation}
    CI = \bar{x} \pm z_{\alpha} \frac{s}{\sqrt{n}}
\end{equation}$$

In [ ]:
alpha = 0.05
CI_amount = st.norm.interval(1 - alpha, loc=mean_amount, scale=st.sem(df_use['amount']))
print(f'CI ({(1-alpha) * 100}%) = {CI_amount}')


### Sobre el número de reclamos por día

Ahora nos gustaría conocer qué proceso modela el número de reclamos por día. Sabemos de antemano que este es un proceso estocástico de conteo; así que una de las opciones de modelado más atractivas es la del proceso de Poisson. Primero observaremos la distribución de nuestros datos.

In [ ]:
claims_per_day = df_use.groupby('date').count()['amount'].values
sample_size = claims_per_day.shape[0]
mean_claims_per_day = np.mean(claims_per_day)
std_claims_per_day = np.std(claims_per_day)
var = np.var(claims_per_day)
print(f'''Estadísticos:
Sample size: {sample_size}
Mean claims per day: {mean_claims_per_day}
Standard deviation: {std_claims_per_day}
Variance: {var}''')


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax = sns.barplot(x=np.arange(0, sample_size), y=claims_per_day, color='red', saturation=0.5)
ax.set_title('Cantidad de siniestros por día')
ax.set_xlabel('Día')
ax.set_ylabel('Cantidad de siniestros')
new_ticks = [i.get_text() for i in ax.get_xticklabels()]
plt.xticks(range(0, len(new_ticks), 20), new_ticks[::20])
plt.show()


#### Prueba de hipótesis sobre la distribución del número de reclamos por día

Para demostrar que este proceso es Poisson homogéneo, aprovecharemos que un proceso de Poisson homogéneo sigue una distribución Poisson. Realizaremos una prueba de bondad de ajuste `chi-square` para determinar si es que esa distribución se ajusta a nuestro proceso estocástico. Usaremos un nivel de significancia del 0.05:

$$\begin{gather*}
    H_o: X \sim \text{Poi}(\lambda t) \\
    H_a: X \not \sim \text{Poi}(\lambda t)
\end{gather*}$$

In [ ]:
test_sample = np.random.poisson(mean_claims_per_day, sample_size)

chisquare(claims_per_day, test_sample, value=0.15)


El `p-value` que hemos calculado no es menor que el nivel de significancia establecido, por lo que modelaremos este proceso estocástico como un proceso de Poisson homogéneo.

#### Prueba de hipótesis sobre la media del número de reclamos por día

Realizamos una prueba de hipótesis sobre la media del número de reclamos, el parámetro que deseamos estimar es también el parámetro que necesita un proceso de Poisson. De nuevo aprovecharemos el Teorema Central del Límite para suponer que esta media sigue una distribución normal, hay que recordar que para este dato tenemos concretamente 365 observaciones. Usaremos un nivel de significancia del 0.05:
$$\begin{gather*}
    H_o: \lambda = \bar{\lambda} \\
    H_a: \lambda \neq \bar{\lambda}
\end{gather*}$$


#### Intervalo de confianza para la media del número de reclamos por día

Aquí también es de nuestro interés conocer un intervalo de confianza para nuestra estimación; seguiremos la misma convención de usar un nivel de significancia del $0.05$, y aprovecharemos el Teorema Central del Límite aplicado a medias para calcular nuestro intervalo de confianza como:
$$\begin{equation}
    CI = \bar{x} \pm z_{\alpha} \frac{s}{\sqrt{n}}
\end{equation}$$

In [ ]:
alpha = 0.05
CI_claims = st.norm.interval(1 - alpha, loc=mean_claims_per_day, scale=st.sem(claims_per_day))
print(f'CI ({(1-alpha) * 100}%): {CI_claims}')


### Independencia entre el monto de reclamos y el número de reclamos

In [ ]:
# Generar una muestra aleatoria de los reclamos
amounts_sample = df_use['amount'].sample(len(claims_per_day), replace=True, random_state=42)
pearsonr(claims_per_day, amounts_sample)

## Simulaciones

In [ ]:
mean_amount = df_use['amount'].describe()['mean']
mean_claims = df_use.groupby('date').count()['amount'].mean()
print(f'Lambda = {mean_amount}')
print(f'Mean claims per day = {mean_claims}')


### Total del reclamo por instante $t$

In [ ]:
def generate_claims(claim_rate: float, mean_amount: float) -> float:
    number_claims = np.random.poisson(claim_rate)
    amount = np.random.exponential(mean_amount, number_claims)
    return np.sum(amount)


### Detectando ruinas

In [ ]:
def test_ruin(T: int, claim_rate: float, mean_amount: float, initial_equity: float, c: float) -> tuple:
    U = np.array([])
    # S1 = generate_claims(claim_rate, mean_amount)
    P = c
    U = np.append(U, initial_equity)

    R = 0
    for i in range(1, T):
        S = generate_claims(claim_rate, mean_amount)
        new_U = U[i-1] + P - S
        U = np.append(U, new_U)
        if U[i] < 0:
            R += 1
            break

    return R, U


### Método de Monte Carlo Crudo

In [ ]:
def ruin_prob(n_sim: int, T: int, claim_rate: float, mean_amount: float, initial_equity: float, c: float, alpha: float) -> tuple:
    ruins = np.array([])
    paths = []

    for _ in range(int(n_sim)):
        R, U = test_ruin(T, claim_rate, mean_amount, initial_equity, c)
        ruins = np.append(ruins, R)
        paths.append(U)
    
    CI = proportion_confint(sum(ruins), n_sim, alpha=alpha, method='normal')

    return np.mean(ruins), paths, CI


### Probabilidad de ruina analítica. Fórmula de Pollaczcek-Khinchin

$$\begin{gather*}
    u := \text{Capital inicial} \\
    \theta := \text{Factor de recargo} \\
    \alpha := \frac{1}{\lambda_Z} \\
    \psi(u) = \frac{1}{1 + \theta} \cdot \exp \left(u \frac{\alpha \theta}{1 + \theta} \right)
\end{gather*}$$

In [ ]:
def pollaczek_khinchin(theta: float, u: float, lam_z: float) -> float:
    alpha = 1 / lam_z
    prob = (1 / (1 + theta)) * np.exp(- alpha * u * (theta / (1 + theta)))
    return prob


In [ ]:
U0 = 1000000
alpha = 0.01
# Factor de recargo
theta = 0.05
c = (1 + theta) * mean_claims * mean_amount
print(f'c = {c}')
T = 120
N_SIM = 100
ruins, paths, CI = ruin_prob(N_SIM, T, mean_claims, mean_amount, U0, c, alpha)
print(f'P(Ruins): {ruins}')
print(f'CI ({(1-alpha) * 100}%): {CI}, Error = {0.5 * (CI[1] - CI[0])}')
p_analytic = pollaczek_khinchin(theta, U0, mean_amount)
print(f'P(Ruins) (analytic): {p_analytic}')


In [ ]:
plt.figure(figsize=(8, 6))
plt.title(f'Simulación del proceso de riesgo \n P(Ruina) = {ruins} \n CI = ({CI[0]:.4f}, {CI[1]:.4f})')
plt.xlabel('Día')
plt.ylabel('Capital')
for path in paths:
    plt.plot(path)

plt.show()


## Probabilidades analíticas

### Variación del factor de recargo

In [ ]:
def test_theta():
    # Variando theta
    pos_theta = np.linspace(0, 1, 100)
    prob = np.array([])
    for theta in pos_theta:
        prob = np.append(prob, pollaczek_khinchin(theta, U0, mean_amount))

    # Visualización de resultados
    plt.figure(figsize=(8, 6))
    plt.title(r'Probabilidad de ruina en función de $\theta$. $U_0 = 1000000$')
    plt.xlabel(r'$\theta$')
    plt.ylabel('Probabilidad de ruina')
    plt.plot(pos_theta, prob, label='Pollaczeck-Khinchin')
    plt.legend()
    plt.show()

test_theta()

### Variación del capital inicial

In [ ]:
def test_u0():
    U = np.linspace(0, 1e7, 100)
    prob = np.array([])
    for u in U:
        prob = np.append(prob, pollaczek_khinchin(theta, u, mean_amount))

    # Visualización de resultados
    plt.figure(figsize=(8, 6))
    plt.title(r'Probabilidad de ruina en función de $U_0$, $\theta = 0.05$')
    plt.xlabel(r'$U_0$')
    plt.ylabel('Probabilidad de ruina')
    plt.plot(U, prob, label='Pollaczeck-Khinchin')
    plt.legend()
    plt.show()

test_u0()